### Sample code - branches count

In [1]:
import pandas as pd
merge = pd.read_csv('data/merge_bq_repo_name.csv') # merge_bq_repo_name is the repo name I merged 
merge.shape

(16467, 2)

In [2]:
merge.head(10)

,repo_name,company_name
0,000webhost/000webhost-sdk-js,000webhost
1,100dayproject/polymer-web-example,100dayproject
2,18F/18f-cf-release,18F
3,18F/api-program,18F
4,18F/caddy-boshrelease,18F
5,18F/cf-abacus,18F
6,18F/cf-meteor-buildpack,18F
7,18F/cg-cf-release,18F
8,18F/cg-style,18F
9,18F/cg-style-gem,18F


In [3]:
merge_repo = list(merge['repo_name']) # get only repo name from dataset
len(merge_repo)

16467

### Create pickle to save data in every iteration

In [4]:
# -- save picle  
import pickle
filepath = 'branch_time.pkl'

def batch(time_list):
    
    with open(filepath, 'ab') as f:
        pickle.dump(time_list, f)

### Scrape data using github API

In [5]:
import requests
import time
import json

auth = ('ChuxinYang','7fa8727d5edc03fe9a4af13c95edf843b7371879')

branch_time = []

for j in range(len(merge_repo)):

    page = 1
    while True: 
        
        # use github API to scrape data
        get = requests.get('https://api.github.com/repos/'+ str(merge_repo[j]) +'/branches?page='+ str(page) +'&per_page=100/access_tokens',
                      auth = auth)
        get_json = json.loads(get.text)
        time.sleep(1) # rate limit
        
        # test if this repo still exist
        if type(get_json) == dict:
            break
            
        else:
            if len(get_json) == 0: # if there is nothing inside, break out of the loop and look for next repo
                break
 
                
            for i in range(len(get_json)):

                repo_tuple = tuple([merge_repo[j]]) # save the repo name first
                repo_tuple += tuple([get_json[i]['name']]) # save the branch name
                repo_tuple += tuple([get_json[i]['commit']['sha']]) # save the sha
                get2 = requests.get('https://api.github.com/repos/'+ str(merge_repo[j]) +'/commits/'+str(get_json[i]['commit']['sha'] + '?page='+ str(page) +'&per_page=100/access_tokens'),auth=auth)
                get2_json = json.loads(get2.text)
                repo_tuple += tuple([get2_json["commit"]["author"]["date"]]) # save the author date
                repo_tuple += tuple([get2_json["commit"]["committer"]["date"]]) # save the committer date first
                batch(repo_tuple) # call batch function to save repo_tuple in a pickle file
                time.sleep(1) # rate limit
                
            page += 1 
    print(j) # print which repo we are scraping
    

0
1
2
3
4
5


### Open pickle file and read data

In [6]:
import pandas as pd
import pickle
colnames = ['repo_name', 'branch_name', 'sha', 'commit_author_date','commit_committer_date'] # set column names

info = []
with open('branch_time.pkl', 'rb') as fr:
    try:
        while True:
            info.append(pickle.load(fr))
    except EOFError:
        pass
    
repo_info = pd.DataFrame(info, columns=colnames)
repo_info

,repo_name,branch_name,sha,commit_author_date,commit_committer_date
0,000webhost/000webhost-sdk-js,master,1fbf6030ffc603848901a499797af545b51de931,2020-01-08T10:55:57Z,2020-01-08T10:55:57Z
1,100dayproject/polymer-web-example,master,56e634379f51e336fab36b3718a2e0dbf4d8f1ee,2016-03-31T02:55:24Z,2016-03-31T02:55:24Z
2,18F/18f-cf-release,bump-uaa,c93512c41a1b5e2a43352ddbb005c33ed9287f43,2017-02-10T14:35:33Z,2017-02-10T14:35:33Z
3,18F/18f-cf-release,master,4b40b8d13d1cfcfb0d81275092f358b3152b0b78,2017-03-21T17:23:16Z,2017-03-21T17:23:16Z
4,18F/18f-cf-release,uaa-warning,84b25e94f3be4cfe55093226fa9ca74d017002d6,2017-03-13T19:54:06Z,2017-03-13T19:54:06Z
5,18F/18f-cf-release,update-even-more-from-upstream,c558e0a2c7f41c17273268322985694d18f5a86a,2016-12-13T15:51:32Z,2016-12-13T15:51:32Z
6,18F/api-program,18f-pages,4e0f798be696950c4476ef273db2e794c85818cf,2017-04-27T17:09:28Z,2017-04-27T17:09:28Z
7,18F/api-program,guides-style,8479d9dcf131e48270a4bb5fcb10d5437aa1b80c,2015-09-02T18:43:42Z,2015-09-02T18:43:42Z
8,18F/api-program,leah-edits2-merge-attempt,4853c7ea9adf61b34e1e8474d410fc000bb7e5a2,2016-08-31T02:37:01Z,2016-08-31T02:37:01Z
9,18F/api-program,leah-updates,e15287200087ce075bba4e1364a3ed72d2e94f20,2016-08-06T18:57:50Z,2016-08-06T18:57:50Z
